# Memetic algorithm

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Постановка задачи

Имеется мультимножество $S$ из $n$ векторов $v \in \mathbb{R}^m$. Необходимо разделить $S$ на $k$ непересекающихся групп $S_1, \dots, S_k$ так, чтобы величина

$$t = \max \limits_{j \in \{1, \dots, m\}} \Big\{|\max \limits_{l \in \{1, \dots, k\}} \left(\sum \limits_{i \in S_l} {v_{ij}} \right) - \min \limits_{l \in \{1, \dots, k\}} \left(\sum \limits_{i \in S_l} {v_{ij}} \right) | \Big\}$$

была минимальной.

## Описание алгоритма

### Репрезентация решения

Сгенерируем исходные данные при $n = 10$, $m = 4$. Пусть $k = 3$. Создадим какое нибудь разбиение для подсчёта целевой функции. Авторы статьи предлагают записывать решение в виде списка `dist` длины n, в каждой позиции котого находятся числа от 1 до $k$, если `dist[i] == l`, то $i$-ый вектор лежит в $l$-ой группе. В соответствии с терминологией генетического программирования список `dist` называется хромосомой, а его элементы – генами. 

In [2]:
n, m, k = 10, 4, 3

In [3]:
np.random.seed(42)
vectors = np.random.random(size=(n, m))
dist = np.random.randint(0, high=k, size=n)
print(vectors, dist, sep='\n\n')

[[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]
 [0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451]
 [0.30424224 0.52475643 0.43194502 0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184]
 [0.45606998 0.78517596 0.19967378 0.51423444]
 [0.59241457 0.04645041 0.60754485 0.17052412]
 [0.06505159 0.94888554 0.96563203 0.80839735]
 [0.30461377 0.09767211 0.68423303 0.44015249]]

[2 0 2 2 1 0 1 1 1 1]


Целевая функция

In [4]:
def obfective_function(vectors, dist):
    sums = np.array([vectors[dist == l].sum(axis=0) for l in range(k)])
    return np.abs(sums.max(axis=0) - sums.min(axis=0)).max()

In [5]:
obfective_function(vectors, dist)

2.5388004515852125

### Первое поколение

Чтобы сгенерировать первое поколение, авторы предлагают два различных способа. Первый заключается в случайной генерации генов с заданными вероятностями (например, равными, как это будет реализовано). Такой подход может привести к большим значениям целевой функции. Второй подход заключается в том, что $q$ векторов разделяются по группам случайным образом, а остальные распределяются жадным алгоритмом. 

Будем для определённости считать, что в начальной популяции $p = 7$ хромосом. 

In [6]:
p = 7

In [7]:
initial_population = np.random.randint(0, high=k, size=(p, n))
print(initial_population)

[[1 1 1 0 2 1 1 1 1 1]
 [1 2 2 1 2 0 1 0 0 1]
 [2 0 1 0 0 0 0 2 0 0]
 [0 2 0 0 2 2 2 0 2 2]
 [0 2 0 1 2 1 0 2 0 1]
 [0 2 2 1 0 2 1 2 2 0]
 [2 0 2 1 2 0 0 1 2 2]]


У каждой хромосомы можно посчитать `fitness value` – значение целевой функции $t$, чтобы сравнивать её с другими хромосомами. В дальнейшем мы будем отбирать хромосомы с "хорошим" значеним $t$, а решения с "плохим" $t$ будем отсеивать. 

In [8]:
def fitness_values(vectors, initial_population):
    return np.array([obfective_function(vectors, dist) for dist in initial_population])

In [9]:
values = fitness_values(vectors, initial_population)
print(np.round(values, decimals=3))

[4.551 1.355 2.793 3.287 2.943 2.484 2.971]


### Генетические операции

#### Скрещивание

Для каждого вновь созданного поколения некоторая часть поколения выбирается, чтобы сгенерировать новое поколение. Операция скрещивания подразумевает способ выбрать двух "родителей". В нашем случае мы выбираем двух родителей бинарным соревнованием. Берутся две случайных хромосомы, сравниваются их `fitness values` и выбирается лучшее. Два соревнования дают нам двух родителей. 

Сама операция кроссовера представляет собой `single point crossover`. Этот метод подразумевает, что выбирается случайная точка $G$ от 1 до $n - 1$ и создаются два "ребёнка". У первого ребёнка все гены до точки $G$ достаются от первого родителя, а гены после точки $G$ – от второго родителя. У второго родителя наоборот. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/OnePointCrossover.svg/231px-OnePointCrossover.svg.png"/>

Выберем двух родителей

In [10]:
np.random.seed(42)
tournament = np.random.choice(np.arange(p), size=(2, 2), replace=False)
parent_1 = initial_population[tournament[0, values[tournament[0]].argmax()]]
parent_2 = initial_population[tournament[1, values[tournament[1]].argmax()]]
print('Папа: {}\nМама: {}'.format(parent_1, parent_2))

Папа: [1 1 1 0 2 1 1 1 1 1]
Мама: [2 0 1 0 0 0 0 2 0 0]


Выберем точку скрещивания

In [11]:
np.random.seed(42)
crossover_point = np.random.randint(1, high=n-1, size=1)
print(crossover_point[0])

7


Процедура скрещивания:

In [12]:
child_1 = np.hstack((parent_1[:crossover_point[0]], parent_2[crossover_point[0]:]))
child_2 = np.hstack((parent_2[:crossover_point[0]], parent_1[crossover_point[0]:]))
print('Сыночек:\t{}\nЛапочка-дочка:\t{}'.format(child_1, child_2))

Сыночек:	[1 1 1 0 2 1 1 2 0 0]
Лапочка-дочка:	[2 0 1 0 0 0 0 1 1 1]


#### Мутация

Операция мутации подразумевает, что каждый ген в хромосоме меняется с заданной вероятностью на случайный. 

In [13]:
np.random.seed(42)
probability = 0.1
mutated = np.where(np.random.random(size=n) <= probability, np.random.randint(0, high=k, size=n), dist)
print('До мутации:\t{}\nПосле мутации:\t{}'.format(dist, mutated))

До мутации:	[2 0 2 2 1 0 1 1 1 1]
После мутации:	[2 0 2 2 1 0 0 1 1 1]
